In [1]:
import os
import json
import requests
from datetime import datetime, timedelta

class NasaClimateProvider:
    def __init__(self):
        self.evapotranspiration_calculator = EvapotranspirationCalculator()
        self.timeout = 60
        self.data_undefined = -999
        self.expiry = timedelta(minutes=60)
        self.start_date = datetime(1981, 1, 1)
        self.end_date = datetime.now()

    def is_cached(self, latitude, longitude):
        path = self.get_cached_path(latitude, longitude)
        return os.path.exists(path)

    def get_custom_climate_data(self, latitude, longitude):
        url = self.get_correct_api_url(latitude, longitude)
        content = self.get_cached_data(latitude, longitude)
        
        if not content:
            print(f"Cached climate data for lat: {latitude}, long: {longitude} not found. Downloading now.")
            try:
                response = requests.get(url, timeout=self.timeout)
                if response.status_code == 200:
                    content = response.text
                    print("NASA API content downloaded successfully.")
                else:
                    raise Exception("NASA API couldn't be reached or connection timed out.")
            except Exception as e:
                print(f"Could not load data from NASA API. Exception thrown: {e}")
                return []

            if content:
                self.cache_data(latitude, longitude, content)
            else:
                print("No content retrieved from API.")
                return []

        data = self.process_data(content)
        return data

    def get_correct_api_url(self, latitude, longitude):
        base_url = "https://power.larc.nasa.gov"
        user_community = "AG"
        parameters = "T2M,PRECTOTCORR,RH2M,ALLSKY_SFC_SW_DWN"
        start_date_str = self.start_date.strftime("%Y%m%d")
        end_date_str = self.end_date.strftime("%Y%m%d")
        url = f"{base_url}/api/temporal/daily/point?parameters={parameters}&community={user_community}&longitude={longitude}&latitude={latitude}&start={start_date_str}&end={end_date_str}&format=JSON"
        return url

    def get_cached_data(self, latitude, longitude):
        path = self.get_cached_path(latitude, longitude)
        if os.path.exists(path):
            file_info = os.stat(path)
            last_access_time = datetime.fromtimestamp(file_info.st_atime)
            if datetime.now() - last_access_time > self.expiry:
                return ""
            else:
                os.utime(path, None)  # Update the access time
                with open(path, 'r') as file:
                    return file.read()
        return ""

    def get_cached_path(self, latitude, longitude):
        temp_dir = os.path.join(tempfile.gettempdir(), f"nasa_climate_data_lat_{latitude}_long_{longitude}")
        return temp_dir

    def cache_data(self, latitude, longitude, content):
        path = self.get_cached_path(latitude, longitude)
        with open(path, 'w') as file:
            file.write(content)

    def process_data(self, content):
        data = json.loads(content)
        properties = data.get('properties', {})
        daily_data = []

        for param, values in properties.items():
            for date_str, value in values.items():
                # Example of processing the data
                data_obj = {
                    'date': datetime.strptime(date_str, "%Y%m%d"),
                    'value': value
                }
                daily_data.append(data_obj)

        return daily_data

# This class needs to be implemented according to the specific methods used for evapotranspiration calculation.
class EvapotranspirationCalculator:
    def calculate_reference_evapotranspiration(self, temperature, solar_radiation, humidity):
        # Placeholder for actual calculation
        return temperature + solar_radiation + humidity

